<img alt="QuantRocket logo" src="https://www.quantrocket.com/assets/img/notebook-header-logo.png">

<a href="https://www.quantrocket.com/disclaimer/">Disclaimer</a>

# Data Collection

The data collection process consists of collecting ARCA ETF listings, creating a universe of leveraged ETFs, then collecting 15 minute bars for the leveraged ETFs.

First, start IB Gateway:

In [1]:
from quantrocket.launchpad import start_gateways
start_gateways(wait=True)

{'ibg1': {'status': 'running'}}

## Collect ETF listings

Then collect the listings:

In [2]:
from quantrocket.master import collect_listings
collect_listings(exchanges="ARCA", sec_types="ETF")

{'status': 'the listing details will be collected asynchronously'}

Monitor flightlog for a completion message:

```
quantrocket.master: INFO Collecting ARCA ETF listings from IB website
quantrocket.master: INFO Requesting details for 2246 ARCA listings found on IB website
quantrocket.master: INFO Saved 1576 ARCA listings to securities master database
```

## Define universe of leveraged ETFs 

Next we download a CSV of several leveraged ETFs:

In [3]:
from quantrocket.master import download_master_file
download_master_file(
    "leveraged_etfs.csv",
    exchanges="ARCA",
    sec_types="ETF", 
    symbols=[
        "SPXL", # DIREXION DLY S&P 500 BULL 3X
        "TNA",  # DIREXION DLY SM CAP BULL 3X
        "ERX",  # DIREXION DLY ENERGY BULL 3X
        "TMF",  # DRX DLY 20+ YR TREAS BULL 3X
        "UPRO", # PROSHARES ULTRAPRO S&P 500
        "DRN",  # DRX DLY REAL ESTATE BULL 3X
        "YINN", # DRX DLY FTSE CHINA BULL 3X
        "UDOW", # PROSHARES ULTRAPRO DOW30
        "URTY", # PROSHARES ULTRAPRO RUSS2000
        "FAS",  # DIREXION DAILY FIN BULL 3X
        "EDC",  # DIREXION DLY EMG MKT BULL 3X
        "RUSL", # DIREXION DLY RUSSIA BULL 3X
        "JNUG", # DIREXION DLY JR GOLD BULL 3X
        "NUGT", # DRX DLY GOLD MINERS BULL 3X
    ])

Then upload the CSV to create the "leveraged-etf" universe:

In [4]:
from quantrocket.master import create_universe
create_universe("leveraged-etf", infilepath_or_buffer="leveraged_etfs.csv", replace=True)

{'code': 'leveraged-etf',
 'provided': 14,
 'inserted': 14,
 'total_after_insert': 14}

## Collect historical data

Next, we create a database for collecting 15-min bars.

> The `shard` parameter is required for intraday databases and determines how to split up large databases into smaller pieces for better performance. Since our universe contains a relatively small number of securities, we can turn this feature off. See the usage guide to learn more.

In [5]:
from quantrocket.history import create_db
create_db("leveraged-etf-15min", universes="leveraged-etf", bar_size="15 mins", shard="off")

{'status': 'successfully created quantrocket.history.leveraged-etf-15min.sqlite'}

Then collect the data:

In [6]:
from quantrocket.history import collect_history
collect_history("leveraged-etf-15min")

{'status': 'the historical data will be collected asynchronously'}

Monitor flightlog for completion:

```
quantrocket.history: INFO [leveraged-etf-15min] Collecting history from IB for 14 securities in leveraged-etf-15min
quantrocket.history: INFO [leveraged-etf-15min] Saved 842150 total records for 14 total securities to quantrocket.history.leveraged-etf-15min.sqlite
```

***

## *Next Up*

Part 2: [Moonshot Backtest](Part2-Moonshot-Backtest.ipynb)